In [1]:
import sys, os

sys.path.append("..")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

/tmp/ipykernel_112186/1745948676.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from src import mhdx_engine, mhdx_extract_stats

In [7]:
indexes_folder = "../datasets/noise_arrays/"

fpath = indexes_folder + "Index_00001.plk"

out = mhdx_extract_stats.load_noise_arrays(fpath)

In [8]:
out

{'sequence': 'HMRDLERLAEELLKKANGDEDWVKEYLKSAGVDPEEAEKLIKAAK',
 'timepoints': array([0.0000e+00, 2.5000e+01, 3.3000e+01, 4.3000e+01, 5.8000e+01,
        7.7000e+01, 1.0200e+02, 1.3500e+02, 1.7900e+02, 2.3700e+02,
        3.1400e+02, 3.1410e+02, 4.1600e+02, 5.5200e+02, 7.3200e+02,
        9.6900e+02, 1.2850e+03, 1.7020e+03, 2.2560e+03, 2.9890e+03,
        3.9610e+03, 5.2480e+03, 6.9540e+03, 9.2140e+03, 1.2209e+04,
        1.6177e+04, 2.1434e+04, 2.8401e+04, 3.7631e+04, 4.9861e+04,
        6.6067e+04, 8.7538e+04]),
 'rates': array([1.97092675e-04, 2.61784306e-04, 3.65676979e-04, 5.80249926e-04,
        7.81488047e-04, 9.28568099e-04, 1.04029065e-03, 1.13967151e-03,
        1.22669748e-03, 1.31220179e-03, 1.39594063e-03, 1.48152793e-03,
        1.56668232e-03, 1.65781380e-03, 1.75004292e-03, 1.84891175e-03,
        1.95551527e-03, 2.07687613e-03, 2.21985814e-03, 2.38021368e-03,
        2.56723640e-03, 2.77764512e-03, 3.04549975e-03, 3.38809776e-03,
        3.83019759e-03, 4.36455141e-03, 5

In [11]:
# Define charge of tensor

charge = 6

In [12]:
engine = mhdx_engine.HDXTimecourseEngine(sequence=out['sequence'],
                            timepoints=out['timepoints'],
                            ks=out['rates'],
                            backexchange=out['backexchange'],
                            charge=charge,
                            rt_center=out['rt'],
                            rt_mean=out['rt'] + out['rt_total_error_mean'],
                            rt_std=out['rt_sigma_baseline'] + out['rt_total_error_std'],
                            dt_center=out['dt_mean_baseline'][charge],
                            dt_mean=out['dt_mean_baseline'][charge] + out['dt_total_error_mean'][charge],
                            dt_std=out['dt_std_baseline'][charge] + out['dt_total_error_std'][charge],
                            ppm_error=out['ppm_errors'],
                            tics=out['intensities'],
                            grid_step_mz=0.01,
                            )

In [13]:
engine.generate_ideal_tensor()
engine.apply_perturbations()
engine.add_random_noise()

In [ ]:
# Which tensor to look at?

In [14]:
tensor = engine.ideal_tensor

In [16]:
def plot_tensor(engine, tensor):

    import matplotlib.pyplot as plt
    import seaborn as sns

    def _three_ticks(n: int):
        # start, middle, end (unique & sorted)
        idx = sorted(set([0, n // 2, n - 1]))
        return idx

    T = tensor.shape[0]
    fig, axes = plt.subplots(
        T, 2, figsize=(7, T * 1.4), dpi=300, constrained_layout=True,
        width_ratios=[3, 1]
    )
    axes = np.atleast_2d(axes)  # makes (T,2) even when T==1

    for i, t_slice in enumerate(tensor):
        # --- left: m/z marginal (sum over RT & DT) ---
        mz_profile = t_slice.sum(axis=(0, 1))  # (M,)
        ax0 = axes[i, 0]
        ax0.plot(engine.mz_labels, mz_profile/np.max(mz_profile), lw=1)
        ax0.set_xlabel("m/z", fontsize=5)
        ax0.set_ylabel("Intensity (a.u.)", fontsize=5)
        ax0.tick_params(axis='both', which='both', labelsize=5, top=False, left=False)
        ax0.spines['top'].set_visible(False)
        ax0.spines['right'].set_visible(False)


        # --- right: RT×DT heatmap (sum over m/z) ---
        rd = t_slice.sum(axis=-1)  # shape (R, D)
        ax1 = axes[i, 1]
        hm = sns.heatmap(rd, cmap="Blues", ax=ax1, cbar=False)

        # build sparse ticks using labels: y=RT (min), x=DT (ms)
        R, D = rd.shape
        xt = _three_ticks(D)
        yt = _three_ticks(R)

        # heatmap cells are centered at i+0.5
        ax1.set_xticks(np.array(xt) + 0.5)
        ax1.set_yticks(np.array(yt) + 0.5)

        ax1.set_xticklabels([f"{engine.dt_labels[j]:.3f}" for j in xt], fontsize=5, rotation=0)
        ax1.set_yticklabels([f"{engine.rt_labels[j]:.2f}" for j in yt], fontsize=5, rotation=0)

        ax1.set_xlabel("DT (ms)", fontsize=5)
        ax1.set_ylabel("RT (min)", fontsize=5)

        ax1.tick_params(axis='both', which='both', labelsize=5, top=False, left=False)
        ax1.spines['top'].set_visible(False)
        ax1.spines['left'].set_visible(False)


In [ ]:
plot_tensor(engine, engine.perturbed_tensor)

In [ ]:
plot_tensor(engine, engine.noisy_tensor)